In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.python.framework import ops

import load_data as load
from models.model import Model
from models.customlayers import *
from models.activations import *
from training import *

import moviepy.editor as mpe
from models.AELSTM import *
L = tf.layers

import matplotlib.pyplot as plt
% matplotlib inline

data_dir = os.path.expanduser('~/Insight/video-representations/frames')

In [2]:
batchsize = 8
sequence_length = 64

model = Model(encoder, lstm_cell, tied_decoder, batchsize, sequence_length)

## LSTM-Encoder Training Graph ##

training_inputs, training_targets = load.inputs('training', batchsize, 1, shuffle=False)

encoded, transitioned, decoded = model.build(training_inputs)
loss = tf.reduce_mean(tf.pow(decoded - training_targets, 2))
fakeloss = tf.reduce_mean(tf.pow(decoded - training_inputs, 2))

optimizer = tf.train.AdamOptimizer(learning_rate=.00001)
trainable_vars = tf.trainable_variables()
clipped_gradients, _ = tf.clip_by_global_norm(tf.gradients(loss, trainable_vars), 1)
train_step = optimizer.apply_gradients(zip(clipped_gradients, trainable_vars))

## LSTM-Encoder Validation Graph ##

validation_inputs, validation_targets = load.inputs('validation', batchsize, 1)

encoded_validation, transitioned_validation, decoded_validation = model.build(validation_inputs, reuse=True)
validation_loss = tf.reduce_mean(tf.pow(decoded_validation - validation_targets, 2))

In [ ]:
saver = tf.train.Saver()
coord = tf.train.Coordinator()

loss_cache = []
fakeloss_cache = []

with tf.Session() as sesh:
    saver.restore(sesh, 'ptypelstm-tied-relu')
    
    try:
        step = 0
        
        while not coord.should_stop():
            loss_val, fakeloss_val = sesh.run([loss, fakeloss])
            
            loss_cache.append(loss_val)
            fakeloss_cache.append(fakeloss_val)
            
    except tf.errors.OutOfRangeError:
        print('Encoder validated: {:.2f}'.format(loss_value))
        
    finally:
        coord.request_stop()
        
    coord.join(threads)

INFO:tensorflow:Restoring parameters from ptypelstm-tied-relu
